In [1]:
import os
import sys; sys.path.insert(0, os.path.abspath("../"))
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction

import multiprocessing as mp
from tqdm import tqdm

In [2]:
data_train = SkinLesion_Dataset(class_task='three_class', partitions=['train']) # default resize as True
data_val = SkinLesion_Dataset(class_task='three_class', partitions=['val'])
preproc = SkinLesionPreprocessing()
cfe = FeaturesExtraction(levels=['global'])
print(f'train: {len(data_train)}',f'val: {len(data_val)}')

train: 5082 val: 1270


In [5]:
def obtain_data(data_sample):

    img = data_sample['img']
    img_preproc = preproc.preprocess(img)
    
    return cfe.extract_features(img_preproc), data_sample['label']

In [ ]:
n_jobs = mp.cpu_count()
features = []
labels = []
with mp.Pool(n_jobs) as pool:
    for result in tqdm(pool.imap(obtain_data, data_train), total= len(data_train)):
        feat, lab = result
        features.append(feat)
        labels.append(lab)

df = pd.DataFrame(features, columns=cfe.features_names)
df['label'] = labels
df.to_feather('../data/three_class/train_all_feat.f') 


  0%|          | 0/5082 [00:00<?, ?it/s]

In [ ]:
n_jobs = mp.cpu_count()
features = []
labels = []
with mp.Pool(n_jobs) as pool:
    for result in tqdm(pool.imap(obtain_data, data_val), total= len(data_val)):
        feat, lab = result
        features.append(feat)
        labels.append(lab)

df = pd.DataFrame(features, columns=cfe.features_names)
df['label'] = labels
df.to_feather('../data/three_class/val_all_feat.f') 

### correct metadata csv DO NOT RUN THIS

In [5]:
def load_images_from_folder(folder):
    images=[]
    for filename in os.listdir(folder):
        full_path = folder + filename
        images.append(full_path)
    return images

In [14]:
path = '../data/three_class/val/mel/'
images_folder = load_images_from_folder(path)
print(np.size(images_folder))
metadata = []

for idx, img_path in tqdm(enumerate(images_folder), total=np.size(images_folder)):
    img_data = {} # img_id, label, size, height, width, path
    img_data['img_id'] = int(img_path.split('/')[-1].split('.')[0][-5:])
    img_data['label'] = 'mel'
    img_data['sublabel'] = img_path.split('/')[-1].split('.')[0][:3]
    image = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_data['size'] = image.shape
    img_data['height'] = image.shape[0]
    img_data['width'] = image.shape[1]
    img_data['path'] = img_path
    img_data['split'] = 'val'
    metadata.append(img_data)

metadata_mel_df = pd.DataFrame(metadata)
del metadata

678


100%|██████████| 678/678 [00:20<00:00, 33.81it/s]


In [9]:
1993+2713+376

5082

In [11]:
md_threeclass_train = pd.concat([metadata_bcc_df, metadata_mel_df, metadata_scc_df], ignore_index=True)
md_threeclass_train

,img_id,label,sublabel,size,height,width,path,split
0,1758,bcc,bcc,"(450, 600, 3)",450,600,../data/three_class/train/bcc/bcc01758.jpg,train
1,782,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00782.jpg,train
2,542,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00542.jpg,train
3,1876,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc01876.jpg,train
4,729,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00729.jpg,train
...,...,...,...,...,...,...,...,...
5077,22,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/scc/scc00022.jpg,train
5078,98,scc,scc,"(450, 600, 3)",450,600,../data/three_class/train/scc/scc00098.jpg,train
5079,348,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/scc/scc00348.jpg,train
5080,244,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/scc/scc00244.jpg,train


In [15]:
94+498+678

1270

In [16]:
md_threeclass_val = pd.concat([metadata_bcc_df, metadata_mel_df, metadata_scc_df], ignore_index=True)
md_threeclass_val

,img_id,label,sublabel,size,height,width,path,split
0,2368,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/bcc/bcc02368.jpg,val
1,2367,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/bcc/bcc02367.jpg,val
2,2145,bcc,bcc,"(450, 600, 3)",450,600,../data/three_class/val/bcc/bcc02145.jpg,val
3,2452,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/bcc/bcc02452.jpg,val
4,2178,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/bcc/bcc02178.jpg,val
...,...,...,...,...,...,...,...,...
1265,450,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00450.jpg,val
1266,399,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00399.jpg,val
1267,418,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00418.jpg,val
1268,406,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00406.jpg,val


In [17]:
md_three_class = pd.concat([md_threeclass_train, md_threeclass_val], ignore_index=True)
md_three_class.to_csv('../data/metadata/three_class.csv')
md_three_class

,img_id,label,sublabel,size,height,width,path,split
0,1758,bcc,bcc,"(450, 600, 3)",450,600,../data/three_class/train/bcc/bcc01758.jpg,train
1,782,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00782.jpg,train
2,542,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00542.jpg,train
3,1876,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc01876.jpg,train
4,729,bcc,bcc,"(1024, 1024, 3)",1024,1024,../data/three_class/train/bcc/bcc00729.jpg,train
...,...,...,...,...,...,...,...,...
6347,450,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00450.jpg,val
6348,399,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00399.jpg,val
6349,418,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00418.jpg,val
6350,406,scc,scc,"(1024, 1024, 3)",1024,1024,../data/three_class/val/scc/scc00406.jpg,val


In [20]:
md_three_class.to_csv('../metadata/three_class.csv')